# Machinations to CadCAD conversion guide 1 
# for Infinite runner model
Shoutout to the:
CADLabs team whose ethereum economic model's (https://github.com/CADLabs/ethereum-economic-model) radcad framework I'm using in this code
cadcad.edu (https://www.cadcad.education/) for their amazing course on cadCAD which I really cant recommend enough

https://machinations.io/docs/tutorials-examples/infinite-runner-game-loop/

## Importing libraries

In [120]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
pd.options.plotting.backend = "plotly"
from dataclasses import dataclass
import copy
from dataclasses import field
from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend

## Defining the types of variables

In [151]:

coffees = int
points = int
assumption = int

## Defining the System Params

In [171]:
#utils
#def default(obj):
#    return field(default_factory=lambda: copy.copy(obj))


@dataclass
class Parameters:
    # crash_chance is the chance of crashing in the beginning
    coffee_addiction_assumption: assumption = 7


# Initialize Parameters instance with default values
system_params = Parameters().__dict__

## State Variables

In [172]:
@dataclass
class StateVariables:
    Total_Coffees_Bought: coffees = 0
    Free_Coffees: coffees = 0
    Current_Points: points = 0
    Lifetime_Points_Redeemed: points = 0

initial_state = StateVariables().__dict__

## Policy Functions

In [187]:
def p_coffee_run(params, substep, state_history, prev_state, **kwargs):
    '''Calculates whether the user will get coffee or not'''

    if (params['coffee_addiction_assumption'] > random.randint(0,10)):
        New_Coffees_Bought = 1 # Coffee Bought
        New_Points = 5
    else:
        New_Coffees_Bought = 0
        New_Points = 0

    return {'New_Coffees_Bought': New_Coffees_Bought,"New_Points":New_Points}






def p_redeem_points(params, substep, state_history, prev_state, **kwargs):
    '''Redeem Points'''

    if prev_state['Current_Points'] > 25:

        New_Points_Redeemed = 25
        New_Coffees_Bought = 1
        New_Free_Coffees = 1
        New_Points = -25
    else:
        New_Points_Redeemed = 0
        New_Coffees_Bought = 0
        New_Free_Coffees = 0
        New_Points = 0
        

    return {'New_Points':New_Points,'New_Points_Redeemed':New_Points_Redeemed ,'New_Coffees_Bought':New_Coffees_Bought ,'New_Free_Coffees': New_Free_Coffees}



## State Update Functions and Partial State Update Blocks

In [190]:

    
def s_update_total_coffees(params, substep, state_history, prev_state, policy_input, **kwargs):
    '''Update the state of Total_Coffees_Bought'''
    
    updated_coffee = np.ceil(prev_state['Total_Coffees_Bought'] + policy_input['New_Coffees_Bought']) 
    return ('Total_Coffees_Bought', max(updated_coffee,0))


            
def s_update_current_points(params, substep, state_history, prev_state, policy_input, **kwargs):
    '''Update the state of the coins variable by the amount of new points generated'''
            
    updated_points = np.ceil(prev_state['Current_Points'] + policy_input['New_Points']+policy_input['New_Points'])
    return ('Current_Points', max(updated_points, 0))
            
          

            
            
def s_update_lifetime_points_redeemed(params, substep, state_history, prev_state, policy_input, **kwargs):
    '''Update the state of Total_Coffees_Bought'''
    
    update_lifetime_points = np.ceil(prev_state['Lifetime_Points_Redeemed'] + policy_input['New_Points_Redeemed'])
    return ('Lifetime_Points_Redeemed', max(update_lifetime_points,0))


            
def s_update_lifetime_free_coffees(params, substep, state_history, prev_state, policy_input, **kwargs):
    '''Update the state of Total_Coffees_Bought'''
    
    update_free_coffees = np.ceil(prev_state['Free_Coffees'] + policy_input['New_Free_Coffees'])
    return ('Free_Coffees', max(update_free_coffees,0))
            
            
            
        

###

state_update_blocks = [
    {
        'policies': {
            'p_redeem_points': p_redeem_points,
            'p_coffee_run': p_coffee_run,

        },
        'variables': {
            'Total_Coffees_Bought': s_update_total_coffees,
            'Free_Coffees': s_update_lifetime_free_coffees,
            'Current_Points':s_update_current_points,
            'Lifetime_Points_Redeemed': s_update_lifetime_points_redeemed

        }
    },

]



In [191]:
# config and run

#number of timesteps
TIMESTEPS = 40
#number of monte carlo runs
RUNS = 5


model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)

experiment = Experiment(simulation)
# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

result = experiment.run()


df = pd.DataFrame(result)
df

Total_Coffees_Bought  Free_Coffees  Current_Points  \
0                     0.0           0.0             0.0   
1                     1.0           0.0            10.0   
2                     2.0           0.0            20.0   
3                     3.0           0.0            30.0   
4                     4.0           1.0             0.0   
..                    ...           ...             ...   
200                  27.0           6.0             0.0   
201                  28.0           6.0            10.0   
202                  29.0           6.0            20.0   
203                  30.0           6.0            30.0   
204                  32.0           7.0             0.0   

     Lifetime_Points_Redeemed  simulation  subset  run  substep  timestep  
0                         0.0           0       0    1        0         0  
1                         0.0           0       0    1        1         1  
2                         0.0           0       0    1        1         2  
3                         0.0           0       0    1        1         3  
4                        25.0           0       0    1        1         4  
..                        ...         ...     ...  ...      ...       ...  
200                     150.0           0       0    5        1        36  
201                     150.0           0       0    5        1        37  
202                     150.0           0       0    5        1        38  
203                     150.0           0       0    5        1        39  
204                     175.0           0       0    5        1        40  

[205 rows x 9 columns]

## Let's plot these 5 monte carlo runs

In [194]:
fig = px.line(
    df,
    x='timestep',
    y=['Total_Coffees_Bought', 'Lifetime_Points_Redeemed'],
    facet_col='run',
    height=500,
    template='seaborn',
    title='Total coffees bought and the lifetime points earned!'
)

fig.update_layout(
    margin=dict(l=20, r=200, t=100, b=20),
)

fig.show()

# STILL NEED TO KEEP DEVELOPING BELLOW!!!

In [164]:
# final coins and distance in every run
df2 = df.groupby(['run'])[['coins','distance']].max()
df2

NameError: name 'df' is not defined

In [165]:
# final coins distance and crash timestep for each run
first_crash = df[df['player_crashes'] == 1].groupby(['run'])['timestep'].min()
df2['crash_timestep'] = first_crash.values
df2


NameError: name 'df' is not defined

## Checking if data matches machinations simulations

In [ ]:
# config and run

#number of timesteps
TIMESTEPS = 100
#number of monte carlo runs
RUNS = 100


model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)

experiment = Experiment(simulation)
# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

result = experiment.run()


df = pd.DataFrame(result)
df

In [ ]:
# final coins and distance in every run
df2 = df.groupby(['run'])[['coins','distance']].max()
# final coins distance and crash timestep for each run
first_crash = df[df['player_crashes'] == 1].groupby(['run'])['timestep'].min()
df2['crash_timestep'] = first_crash.values
df2.describe()

In [ ]:
# comparing with our machinations simulation results
machinations_results = pd.read_csv('data\inifite_runner_machinations100.csv')

# Data cleaning and processing
array=[]
for i in machinations_results['play'].unique():
    if i ==1:
        pass
    else:
        array.append(dict(machinations_results[machinations_results['play']==i].iloc[-1]))

machinations_df = pd.DataFrame(array)
machinations_df[['Coins / 9','Distance / 8', 'step']].describe()
machinations_df.rename(columns = {'Coins / 9':'coins', 'Distance / 8':'distance',
                              'step':'crash_timestep'}, inplace = True)


In [ ]:
machinations_df.head(10)

In [ ]:
# Means of coins distance and crash_timestep column in CadCAD simulation

df2[['coins','distance','crash_timestep']].mean()

In [ ]:
# Means of coins distance and crash_timestep column in Machinations simulation

machinations_df[['coins','distance','crash_timestep']].mean()

In [ ]:
error = df2[['coins','distance','crash_timestep']].mean() - machinations_df[['coins','distance','crash_timestep']].mean()

The means of all columns for 100 runs look similar so it seems like we succesfully converted the model

# Analysis

### Let's check when the first crash happens for each subset and run

In [ ]:
first_crash = df[df['player_crashes'] == 1].groupby(['subset', 'run'])['timestep'].min().reset_index()

In [ ]:
fig = px.bar(first_crash, x='run', y='timestep', facet_col='subset', title='Player crashed timestep by Run')
fig.show()


In [ ]:
fig = px.histogram(first_crash, x='timestep', title='Player crashed timestep histogram', nbins=30)
fig.show()

In [ ]:
first_crash

### Let's run a parameter sweep on crash chance from 10% to 60% and see how 20 monte carlo runs on each play out

In [ ]:


# config and run
TIMESTEPS = 50
RUNS = 6
model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)
simulation.model.params.update({
    # Running a parameter sweep on the initial chance of crashing 
    'crash_chance':[1,5,10,20,30,40,50,60,70]
})
experiment = Experiment(simulation)

# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

result = experiment.run()


df = pd.DataFrame(result)
df

In [ ]:
# checking when the first crash happened in each subset and run
first_crash = df[df['player_crashes'] == 1].groupby(['subset', 'run'])['timestep'].min().reset_index()

In [ ]:
# plotting the timestep for each crash
fig = px.bar(first_crash, x='run', y='timestep', facet_col='subset', title='Player crashed timestep by Subset and Run')
fig.show()

As we increase the crash chance its clear the timestep at which on average seems to decrease let's prove this by calculating the mean

In [ ]:
subset_means = first_crash.groupby('subset')['timestep'].mean()

# Create bar chart
fig = px.bar(subset_means, x=subset_means.index, y=subset_means.values, title='Mean crash timestep', height=500)

# Create line graph
line_trace = go.Scatter(x=subset_means.index, y=subset_means.values, mode='lines', name='Mean First Change')

# Add line trace to bar chart
fig.add_trace(line_trace)

fig.update_layout(
    margin=dict(l=100, r=20, t=100, b=20),
)
# Show plot
fig.show()



In [ ]:
fig = px.line(
    df,
    x='timestep',
    y=['coins', 'distance'],
    facet_row='subset',
    facet_col='run',
    height=800,
    template='seaborn',
    title='Distance and coins plotted for different runs and subsets'
)

fig.update_layout(
    margin=dict(l=100, r=100, t=100, b=100),
)

fig.show()

In [ ]:
# on average when does the crash happen for each run
first_crash = df[df['player_crashes'] == 1].groupby(['subset', 'run'])['timestep'].min().reset_index()
subset_means = first_crash.groupby('subset')['timestep'].mean()
subset_means

In [ ]:
descriptive_stats = df.groupby(['subset']).agg(['mean', 'median', 'std', 'min', 'max'])
descriptive_stats['distance']

In [ ]:
descriptive_stats['coins']

## Scenario Analysis

Let's say the game team has a shop in the game where certain items cost 50 coins. And they believe that the median player should have to play 10 games to earn these items to adequently incentivize them. What should be the initial value of the crash chance parameter to get 5 as the median coins

In [ ]:
TIMESTEPS = 30
RUNS = 100

model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)

def median_coin_100_runs(i):
    simulation.model.params.update({
        # Running a parameter sweep on the initial chance of crashing 
        'crash_chance':[i]
    })
    experiment = Experiment(simulation)

    # Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
    experiment.engine = Engine(backend=Backend.PATHOS)

    result = experiment.run()


    df = pd.DataFrame(result)

    median_coins = df['coins'].median()
    return median_coins



In [ ]:
median_coins_expected = 5
crash_chance_initial_guess = 1
step_distance = 3


guess = crash_chance_initial_guess

for i in range(20):
    median_for_guess = median_coin_100_runs(guess)
    print(i,', crash chance = ',guess, ', median_for_guess = ', median_for_guess)
    if median_for_guess>median_coins_expected:
        guess+=step_distance
    else:
        break

print(guess)

In [ ]:
guess 

Therefore to get a median coin value of 5 we need crash_chance parameter = 22